In [2]:
pip uninstall tensorflow_model_analysis tensorflow -y

Found existing installation: tensorflow-model-analysis 0.26.0
Uninstalling tensorflow-model-analysis-0.26.0:
  Successfully uninstalled tensorflow-model-analysis-0.26.0
Found existing installation: tensorflow 2.3.0
Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow_model_analysis==0.26.0 tensorflow==2.3 

  Using cached tensorflow_model_analysis-0.26.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.3.0-cp37-cp37m-macosx_10_11_x86_64.whl (165.1 MB)
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.0 which is incompatible.
tensorflow 2.3.0 requires scipy==1.4.1, but you'll have scipy 1.6.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# This setup was tested with TF 2.3 and TFMA 0.24 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))

print('Installing Tensorflow Model Analysis and Dependencies')
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

TF version: 2.3.0
Installing Tensorflow Model Analysis and Dependencies
Beam version: 2.28.0
TFMA version: 0.26.0


In [2]:
# Download the tar file from GCP and extract it
import io, os, tempfile
TAR_NAME = 'saved_models-2.2'
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, TAR_NAME, 'data')
MODELS_DIR = os.path.join(BASE_DIR, TAR_NAME, 'models')
SCHEMA = os.path.join(BASE_DIR, TAR_NAME, 'schema.pbtxt')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

!tar xf {TAR_NAME}.tar
!mv {TAR_NAME} {BASE_DIR}

print("Here's what we downloaded:")
!ls -R {BASE_DIR}

Here's what we downloaded:
saved_models-2.2

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2:
data         models       schema.pbtxt

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/data:
eval  train

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/data/eval:
data.csv

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/data/train:
data.csv

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/models:
estimator keras

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/models/estimator:
eval_model_dir    serving_model_dir

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/models/estimator/eval_model_dir:
1591221811

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/saved_models-2.2/models/estimator/eval_model_dir/1591221811:
saved_model.pb tmp.pbtxt      variables

/var/folders/5z/6y

In [3]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

In [4]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if feature.type == schema_pb2.FLOAT:
      example.features.feature[key].float_list.value[:] = (
          [float(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.INT:
      example.features.feature[key].int64_list.value[:] = (
          [int(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.BYTES:
      example.features.feature[key].bytes_list.value[:] = (
          [line[key].encode('utf8')] if len(line[key]) > 0 else [])
  # Add a new column 'big_tipper' that indicates if tips was > 20% of the fare. 
  # TODO(b/157064428): Remove after label transformation is supported for Keras.
  big_tipper = float(line['tips']) > float(line['fare']) * 0.2
  example.features.feature['big_tipper'].float_list.value[:] = [big_tipper]
  examples.append(example)

tfrecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())

!ls {tfrecord_file}

/var/folders/5z/6yzfsx8163d7bztm4gknfy9c0000gp/T/tmpijgaz3iv/train_data.rio


In [5]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
keras_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models) we need to add a `label_key`.
    label_key: "big_tipper"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our keras model.
keras_model_path = os.path.join(MODELS_DIR, 'keras', '2')
keras_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=keras_model_path,
    eval_config=keras_eval_config)

keras_output_path = os.path.join(OUTPUT_DIR, 'keras')

# Run TFMA
keras_eval_result = tfma.run_model_analysis(
    eval_shared_model=keras_eval_shared_model,
    eval_config=keras_eval_config,
    data_location=tfrecord_file,
    output_path=keras_output_path)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [6]:
eval_result = keras_eval_result

In [ ]:
tfma.view.render_slicing_metrics(eval_result, slicing_column='trip_start_hour')

In [ ]:
tfma.view.render_slicing_metrics(eval_result, slicing_column='trip_start_day')

In [ ]:
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['trip_start_hour', 'trip_start_day']))